Baixando e extraindo as imagens da base siim_small

In [ ]:
!wget https://s3.amazonaws.com/fast-ai-imagelocal/siim_small.tgz
!tar -xvf siim_small.tgz

--2025-05-09 01:21:19--  https://s3.amazonaws.com/fast-ai-imagelocal/siim_small.tgz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.248.134, 3.5.23.165, 54.231.139.128, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.248.134|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33276453 (32M) [application/x-tar]
Saving to: ‘siim_small.tgz’

siim_small.tgz      100%[===================>]  31.73M  31.2MB/s    in 1.0s    

2025-05-09 01:21:20 (31.2 MB/s) - ‘siim_small.tgz’ saved [33276453/33276453]

siim_small/
siim_small/train/
siim_small/train/No Pneumothorax/
siim_small/train/No Pneumothorax/000039.dcm
siim_small/train/No Pneumothorax/000091.dcm
siim_small/train/No Pneumothorax/000192.dcm
siim_small/train/No Pneumothorax/000176.dcm
siim_small/train/No Pneumothorax/000162.dcm
siim_small/train/No Pneumothorax/000215.dcm
siim_small/train/No Pneumothorax/000156.dcm
siim_small/train/No Pneumothorax/000008.dcm
siim_small/train/No Pneumothorax/000037.d

Instalando as bibliotecas necessárias

In [ ]:
!pip install pydicom opencv-python scikit-image

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 16.9 MB/s eta 0:00:00


Imports e definições

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from pydicom import dcmread
from pathlib import Path
from __future__ import print_function
from __future__ import division
from sklearn.metrics import confusion_matrix
import cv2
import math

# define os comparadores do OpenCV
metodos = {
    "CORREL": cv2.HISTCMP_CORREL,
    "CHISQR": cv2.HISTCMP_CHISQR,
    "INTERSECT": cv2.HISTCMP_INTERSECT,
    "BHATTACHARYYA": cv2.HISTCMP_BHATTACHARYYA
}

# define os caminhos das pastas
pneumo = Path("/content/siim_small/train/Pneumothorax")
no_pneumo = Path("/content/siim_small/train/No Pneumothorax")

Carregando e listando as imagens e respectivos histogramas

In [ ]:
# lê os 15 primeiros de cada pasta
pneumo_imagens = sorted(pneumo.glob("*.dcm"))[:15]
no_pneumo_imagens = sorted(no_pneumo.glob("*.dcm"))[:15]

pneumo_datasets = [dcmread(i) for i in pneumo_imagens]
no_pneumo_datasets = [dcmread(i) for i in no_pneumo_imagens]

# junta as duas listas em uma só
datasets = pneumo_datasets + no_pneumo_datasets

# calcula histogramas para todas as imagens
# obs.: o .append adiciona o resultado direto na lista
hists = []
for imagem in datasets:
  hists.append(cv2.calcHist([imagem.pixel_array], [0], None, [256], [0, 256]))

Cria um dicionário onde a chave é o nome do método ("CORREL", "CHISQR" etc.) e o valor é uma lista vazia onde a gente vai guardar os pares (label_real, label_predita).

In [ ]:
# lista de rótulos pra usar pra saber quem é o que
# 0 para No Pneumothorax (negativo), 1 para Pneumothorax (positivo)
labels = [1]*15 + [0]*15

# dicionários pra guardar métricas
resultados = {nome_metodo: [] for nome_metodo in metodos}

Fazendo todas as comparações

In [ ]:
for indice_teste in range(len(hists)):
    # seleciona o histograma e o rótulo da imagem que será usada como teste
    histograma_teste = hists[indice_teste]
    rotulo_teste = labels[indice_teste]

    # cria uma base de treino removendo a imagem de teste
    histogramas_treino = [h for i, h in enumerate(hists) if i != indice_teste]
    rotulos_treino = [r for i, r in enumerate(labels) if i != indice_teste]

    # para cada método de comparação de histograma
    for nome_metodo, metodo_cv2 in metodos.items():
        # lista para guardar as similaridades (ou distâncias) entre o teste e cada treino
        similaridades = []

        # compara a imagem de teste com cada uma da base de treino
        for histograma_treino in histogramas_treino:
            pontuacao = cv2.compareHist(histograma_teste, histograma_treino, metodo_cv2)
            similaridades.append(pontuacao)

        # determina qual imagem da base de treino é a mais parecida
        if nome_metodo in ["CORREL", "INTERSECT"]:
            # nestes métodos, quanto maior a pontuação, mais parecidos
            indice_melhor = np.argmax(similaridades)
        else:
            # nestes métodos, quanto menor a pontuação, mais parecidos
            indice_melhor = np.argmin(similaridades)

        # pega o rótulo da imagem mais parecida
        rotulo_predito = rotulos_treino[indice_melhor]

        # salva no dicionário o rótulo real e o predito
        resultados[nome_metodo].append((rotulo_teste, rotulo_predito))

Calculando e imprimindo as métricas

In [ ]:
# para cada método de comparação
for nome_metodo in resultados:
     # separa os rótulos verdadeiros e preditos
    verdadeiros, preditos = zip(*resultados[nome_metodo])

    # calcula a matriz de confusão: [ [VP, FN], [FP, VN] ]
    matriz = confusion_matrix(verdadeiros, preditos, labels=[1, 0])
    VP, FN = matriz[0]
    FP, VN = matriz[1]

    # calcula sensibilidade (recall) e especificidade
    sensibilidade = VP / (VP + FN) if (VP + FN) else 0
    especificidade = VN / (VN + FP) if (VN + FP) else 0

    # exibe resultados
    print(f"\n===== {nome_metodo} =====")
    print(f"\nMatriz de confusão:\n{matriz}")
    print(f"Sensibilidade: {sensibilidade:.2f}")
    print(f"Especificidade: {especificidade:.2f}\n")


===== CORREL =====

Matriz de confusão:
[[8 7]
 [7 8]]
Sensibilidade: 0.53
Especificidade: 0.53


===== CHISQR =====

Matriz de confusão:
[[9 6]
 [9 6]]
Sensibilidade: 0.60
Especificidade: 0.40


===== INTERSECT =====

Matriz de confusão:
[[ 5 10]
 [ 7  8]]
Sensibilidade: 0.33
Especificidade: 0.53


===== BHATTACHARYYA =====

Matriz de confusão:
[[6 9]
 [9 6]]
Sensibilidade: 0.40
Especificidade: 0.40

